## Reads in H1 DNN training and generates an output root TTree with DDN output added


In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from matplotlib import rc
from numpy import inf
import os

from os import listdir


import uproot3

import matplotlib as mpl

from datetime import datetime
import subprocess

mpl.rcParams.update({'font.size': 19})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 



In [2]:
has_gpu = True

#has_gpu = True

regression_training_name = 'training_h1_reg_v2a'

output_root_file = 'dnn-output-h1.root'

input_file = '/data/owen/DIS-reco/h1-fullsim-2021-09-27-v2a/all-h1-rapgap.root'




#-- for testing
#max_events =   10000

#-- for all events
max_events = 1e9







### Read in the input TTree.

The input ttree variables should be fairly self explanatory from the names.

The obs_x, obs_y, and obs_Q2 are arrays that contain the following calculations:

0 - electron

1 - E0 E Sigma

2 - E0 Theta Sigma

3 - DA

4 - hadron

5 - ISigma

6 - IDA

7 - Theta Sigma gamma

8 - eSigma

They are there for convenience to compare the DNN predictions with the standard methods.  They are not inputs to the DNN, so don't worry about them if you don't need or want them.


In [3]:
%%time


ur_file = uproot3.open(input_file)

print (ur_file.keys()) 
ur_tree = ur_file['Rapgap/minitree']
print(ur_tree)
ur_tree.show()


pandas_df   =  ur_tree.pandas.df(
    ['has_isr','has_fsr',
     'tower_sum_40','n_towers_40', ''
     'eta_pho_closest_to_ebeam','e_pho_closest_to_ebeam', 'phi_pho_closest_to_ebeam',
     'obs_x', 'obs_y', 'obs_Q2',
     'from_tlv_gen_Q2','from_tlv_gen_x','from_tlv_gen_y',
     'obs_e_e','obs_e_pz','obs_e_pt','obs_e_phi',
     'obs_hfs_e','obs_hfs_pz','obs_hfs_pt','obs_hfs_phi',
     'obs_dphi',
     'Empz', 'obs_e_trk_e',
     'gen_eUncomb_E','gen_eUncomb_theta',
     'gen_HFS_Sigma','gen_HFS_T',
     'beam_e_e', 'beam_p_e',
     'wgt'
    ],
    entrystop=max_events,flatten=True)

print('\n\n Number of entries in pandas_df:  %d ' % pandas_df.shape[0] )

[b'Rapgap;1']
<TTree b'minitree' at 0x7fa9df6c1910>
wgt                        (no streamer)              asdtype('>f4')
Empz                       (no streamer)              asdtype('>f4')
from_tlv_gen_Q2            (no streamer)              asdtype('>f4')
from_tlv_gen_x             (no streamer)              asdtype('>f4')
from_tlv_gen_y             (no streamer)              asdtype('>f4')
gen_new_Qi2s               (no streamer)              asdtype('>f4')
gen_new_xis                (no streamer)              asdtype('>f4')
gen_new_yis                (no streamer)              asdtype('>f4')
gen_new_Q2ida              (no streamer)              asdtype('>f4')
gen_new_xida               (no streamer)              asdtype('>f4')
gen_new_yida               (no streamer)              asdtype('>f4')
gen_HFS_Sigma              (no streamer)              asdtype('>f4')
gen_HFS_T                  (no streamer)              asdtype('>f4')
gen_eUncomb_E              (no streamer)           

### Add any derived variables here

In [4]:
%%time

pandas_df.eval( 'obs_hfs_Empz = obs_hfs_e - obs_hfs_pz', inplace=True )
pandas_df.eval( 'obs_e_Empz = obs_e_e - obs_e_pz', inplace=True )

pandas_df.eval( 'obs_event_Empz = obs_hfs_Empz + obs_e_Empz', inplace=True )

pandas_df.eval( 'rot_pt1 = 0.70710678 * obs_hfs_pt - 0.70710678 * obs_e_pt', inplace=True )
pandas_df.eval( 'rot_pt2 = 0.70710678 * obs_hfs_pt + 0.70710678 * obs_e_pt', inplace=True )

pandas_df.eval( 'rot_Empz1 = 0.70710678 * obs_hfs_Empz - 0.70710678 * obs_e_Empz', inplace=True )
pandas_df.eval( 'rot_Empz2 = 0.70710678 * obs_hfs_Empz + 0.70710678 * obs_e_Empz', inplace=True )

pandas_df.eval( 'gen_log_x = log(from_tlv_gen_x)', inplace=True )
pandas_df.eval( 'gen_log_y = log(from_tlv_gen_y)', inplace=True )
pandas_df.eval( 'gen_log_Q2 = log(from_tlv_gen_Q2)', inplace=True )

pandas_df.eval( 'e_ecal_over_trk_ratio = tower_sum_40/obs_e_trk_e', inplace=True )
pandas_df.eval( 'e_ecal_over_trk_ratio = (e_ecal_over_trk_ratio<4)*e_ecal_over_trk_ratio + (e_ecal_over_trk_ratio>4)*4', inplace=True )

pandas_df.eval( 'dphi_pho_closest_to_ebeam = obs_e_phi - phi_pho_closest_to_ebeam', inplace=True )
pandas_df.eval( 'dphi_pho_closest_to_ebeam = (abs(dphi_pho_closest_to_ebeam)<3.14159265)*(dphi_pho_closest_to_ebeam)+(dphi_pho_closest_to_ebeam>3.14159265)*(dphi_pho_closest_to_ebeam-2*3.14159265) + (dphi_pho_closest_to_ebeam<-3.14159265)*(dphi_pho_closest_to_ebeam+2*3.14159265)', inplace=True )
pandas_df.eval( 'dphi_pho_closest_to_ebeam = (dphi_pho_closest_to_ebeam>0)*dphi_pho_closest_to_ebeam + (dphi_pho_closest_to_ebeam<0)*(dphi_pho_closest_to_ebeam+2*3.14159265)', inplace=True )
pandas_df.eval( 'dphi_pho_closest_to_ebeam = (phi_pho_closest_to_ebeam!=0)*(dphi_pho_closest_to_ebeam)+(phi_pho_closest_to_ebeam==0)*(-1)', inplace=True )

pandas_df.eval( 'e_pho_closest_to_ebeam = (e_pho_closest_to_ebeam<30)*e_pho_closest_to_ebeam + (e_pho_closest_to_ebeam>30)*30', inplace=True )

pandas_df.eval( 'n_towers_40 = (n_towers_40<7)*n_towers_40 + (n_towers_40>=7)*7', inplace=True  )

pandas_df.eval( 'has_norad = (has_isr==0) and (has_fsr==0)', inplace=True )

CPU times: user 5.48 s, sys: 3.19 s, total: 8.67 s
Wall time: 5.23 s


In [5]:
%%time

pandas_df.eval( 'gen_eBare_pt = gen_eUncomb_E * sin(gen_eUncomb_theta)', inplace=True )
pandas_df.eval( 'gen_eBare_pz = gen_eUncomb_E * cos(gen_eUncomb_theta)', inplace=True )
pandas_df.eval( 'gen_eBare_Sig_e = gen_eUncomb_E - gen_eBare_pz', inplace=True )


pandas_df.eval( 'gen_ptbal = 1. - gen_eBare_pt/gen_HFS_T', inplace=True )
pandas_df.eval( 'gen_pzbal = 1. - (gen_eBare_Sig_e + gen_HFS_Sigma)/(2.*beam_e_e)', inplace=True )

pandas_df.eval( 'gen_empz = gen_eBare_Sig_e + gen_HFS_Sigma', inplace=True )

pandas_df.eval( 'obs_ptbal = 1. - obs_e_pt / obs_hfs_pt', inplace=True )
pandas_df.eval( 'obs_pzbal = 1. - (obs_hfs_Empz + obs_e_Empz)/2./beam_e_e', inplace=True )

pandas_df.eval( 'obs_hfs_theta = arctan2(obs_hfs_pt,obs_hfs_pz)', inplace=True )


CPU times: user 2.87 s, sys: 1e+03 ms, total: 3.87 s
Wall time: 1.65 s


## Apply any event selection here.

In [6]:
%%time

print('\n\n Number of entries in pandas_df before selection :  %d ' % pandas_df.shape[0] )


pandas_df = pandas_df.query('Empz > 0')


pandas_df = pandas_df.query('obs_event_Empz > 46 and obs_event_Empz < 62')

pandas_df = pandas_df.query('gen_empz > 44 and gen_empz < 64')

pandas_df = pandas_df.query('obs_hfs_pt > 0')

pandas_df = pandas_df.query('from_tlv_gen_Q2 > 200')

pandas_df = pandas_df.query('e_ecal_over_trk_ratio > 0')

print('\n\n Number of entries in pandas_df after selection:  %d ' % pandas_df.shape[0] )







 Number of entries in pandas_df before selection :  61636821 


 Number of entries in pandas_df after selection:  12030185 
CPU times: user 8.15 s, sys: 4.71 s, total: 12.9 s
Wall time: 12.7 s


## Set up machine learning stuff

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from pickle import load

#-- Ben suggested to try this.  2021-08-07
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(patience=10,
               verbose=True,
               restore_best_weights=True)

import os

print(tf.config.list_physical_devices())

if has_gpu :
    os.environ['CUDA_VISIBLE_DEVICES']="0"
    physical_devices = tf.config.list_physical_devices('GPU') 
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

#####physical_devices = tf.config.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Load the inputs for the DNN and transform them.   Don't change anything here!

In [8]:

X = np.c_[
    pandas_df['e_ecal_over_trk_ratio'].to_numpy(),
    pandas_df['n_towers_40'].to_numpy(),
    pandas_df['eta_pho_closest_to_ebeam'].to_numpy(),
    pandas_df['e_pho_closest_to_ebeam'].to_numpy(),
    pandas_df['dphi_pho_closest_to_ebeam'].to_numpy(),
    pandas_df['obs_e_pz'].to_numpy(),
    pandas_df['obs_e_e'].to_numpy(),
    pandas_df['obs_hfs_pz'].to_numpy(),
    pandas_df['obs_hfs_e'].to_numpy(),
    pandas_df['rot_pt1'].to_numpy(),
    pandas_df['rot_Empz1'].to_numpy(),
    pandas_df['rot_pt2'].to_numpy(),
    pandas_df['obs_pzbal'].to_numpy(),
    pandas_df['obs_ptbal'].to_numpy(),
    pandas_df['obs_dphi'].to_numpy(),
]


#-- Load the scaler transformations!  These are essential when reusing the training with a different dataset.

scaler = load( open('%s-scalers/input_scaler.pkl' % regression_training_name, 'rb'))
X = scaler.transform(X)

    

## Set up the regression network

In [9]:
model_r = tf.keras.models.load_model('%s_regression' % regression_training_name )
model_r.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1024      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               66048     
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              5

In [10]:
%%time

mypreds_r = model_r.predict(X,batch_size=1000)

CPU times: user 9.32 s, sys: 988 ms, total: 10.3 s
Wall time: 7.25 s


### Undo the variable transformations

In [11]:
scalerY = load( open('%s-scalers/target_scaler.pkl' % regression_training_name , 'rb'))

inv_trans_pred = scalerY.inverse_transform(mypreds_r)
pred_vals = np.exp( inv_trans_pred )

### Save the results in an output root file

You may only care about the first 6 branches here (gen and dnn x,y,Q2).

The other x, y, and Q2 calculations are included for ease of comparison and can be omitted if you don't need or want them.

In [12]:
%%time


root_file = uproot3.recreate( output_root_file )

root_file['dnnout'] = uproot3.newtree(
        {
            'gen_x': np.float32,
            'gen_y' : np.float32,
            'gen_q2' : np.float32,
            'dnn_x' : np.float32,
            'dnn_y' : np.float32,
            'dnn_q2' : np.float32,
            'has_isr' : bool,
            'has_fsr' : bool,
            'has_norad' : bool,
            'rec_x_e' : np.float32,
            'rec_y_e' : np.float32,
            'rec_q2_e' : np.float32,
            'rec_x_da' : np.float32,
            'rec_y_da' : np.float32,
            'rec_q2_da' : np.float32,
            'rec_x_h' : np.float32,
            'rec_y_h' : np.float32,
            'rec_q2_h' : np.float32,
            'rec_x_is' : np.float32,
            'rec_y_is' : np.float32,
            'rec_q2_is' : np.float32,
            'wgt' :np.float32,
        }
    )

root_file['dnnout'].extend(
        {
            'gen_x'  : pandas_df['from_tlv_gen_x'].to_numpy(),
            'gen_y'  : pandas_df['from_tlv_gen_y'].to_numpy(),
            'gen_q2' : pandas_df['from_tlv_gen_Q2'].to_numpy(),
            'dnn_x'  : pred_vals[:,0],
            'dnn_y'  : pred_vals[:,2],
            'dnn_q2' : pred_vals[:,1],
            'has_isr'   : pandas_df['has_isr'].to_numpy(),
            'has_fsr'   : pandas_df['has_fsr'].to_numpy(),
            'has_norad'   : pandas_df['has_norad'].to_numpy(),
            'rec_x_e'  : pandas_df['obs_x[0]'].to_numpy(),
            'rec_y_e'  : pandas_df['obs_y[0]'].to_numpy(),
            'rec_q2_e' : pandas_df['obs_Q2[0]'].to_numpy(),
            'rec_x_da'  : pandas_df['obs_x[3]'].to_numpy(),
            'rec_y_da'  : pandas_df['obs_y[3]'].to_numpy(),
            'rec_q2_da' : pandas_df['obs_Q2[3]'].to_numpy(),
            'rec_x_h'  : pandas_df['obs_x[4]'].to_numpy(),
            'rec_y_h'  : pandas_df['obs_y[4]'].to_numpy(),
            'rec_q2_h' : pandas_df['obs_Q2[4]'].to_numpy(),
            'rec_x_is'  : pandas_df['obs_x[5]'].to_numpy(),
            'rec_y_is'  : pandas_df['obs_y[5]'].to_numpy(),
            'rec_q2_is' : pandas_df['obs_Q2[5]'].to_numpy(),
            'wgt' : pandas_df['wgt'].to_numpy(),
        }
    )

root_file.close()

CPU times: user 25.3 s, sys: 959 ms, total: 26.3 s
Wall time: 26.4 s
